In [1]:
# 3. Google Drive Bağlantısı
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
torch.cuda.empty_cache()


In [3]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

batch_size = 8

# Verisetinizin bulunduğu yol
data_dir = '/content/drive/MyDrive/Dataset'

# Veri setindeki tüm görüntülerin ön işlenmesi
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 384x384 boyutuna getirme
    transforms.ToTensor(),  # Tensor'e dönüştürme
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalizasyon
])


# Veri setlerini yükleme
train_dataset = datasets.ImageFolder(root=f"{data_dir}/train", transform=transform)
val_dataset = datasets.ImageFolder(root=f"{data_dir}/validation", transform=transform)
test_dataset = datasets.ImageFolder(root=f"{data_dir}/test", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

In [4]:
import timm
import torch

# Tiny ViT modelini yükleme
model = timm.create_model('vit_tiny_patch16_224', pretrained=True)  # pre-trained model




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/22.9M [00:00<?, ?B/s]

In [5]:
models = timm.list_models()
print(models)

['bat_resnext26ts', 'beit_base_patch16_224', 'beit_base_patch16_384', 'beit_large_patch16_224', 'beit_large_patch16_384', 'beit_large_patch16_512', 'beitv2_base_patch16_224', 'beitv2_large_patch16_224', 'botnet26t_256', 'botnet50ts_256', 'caformer_b36', 'caformer_m36', 'caformer_s18', 'caformer_s36', 'cait_m36_384', 'cait_m48_448', 'cait_s24_224', 'cait_s24_384', 'cait_s36_384', 'cait_xs24_384', 'cait_xxs24_224', 'cait_xxs24_384', 'cait_xxs36_224', 'cait_xxs36_384', 'coat_lite_medium', 'coat_lite_medium_384', 'coat_lite_mini', 'coat_lite_small', 'coat_lite_tiny', 'coat_mini', 'coat_small', 'coat_tiny', 'coatnet_0_224', 'coatnet_0_rw_224', 'coatnet_1_224', 'coatnet_1_rw_224', 'coatnet_2_224', 'coatnet_2_rw_224', 'coatnet_3_224', 'coatnet_3_rw_224', 'coatnet_4_224', 'coatnet_5_224', 'coatnet_bn_0_rw_224', 'coatnet_nano_cc_224', 'coatnet_nano_rw_224', 'coatnet_pico_rw_224', 'coatnet_rmlp_0_rw_224', 'coatnet_rmlp_1_rw2_224', 'coatnet_rmlp_1_rw_224', 'coatnet_rmlp_2_rw_224', 'coatnet_rmlp_2

In [6]:
import torch.nn as nn
import torch.optim as optim

# Son katmanı sınıf sayınıza göre değiştirme
num_classes = len(train_dataset.classes)
model.head = nn.Linear(model.head.in_features, num_classes)

# Kayıp fonksiyonu ve optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Modeli CUDA'ya taşıma (GPU kullanıyorsanız)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


In [ ]:
from tqdm.auto import tqdm

# Eğitim fonksiyonu
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0

    total = 0
    for inputs, labels in  tqdm(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return running_loss / len(train_loader), accuracy

# Eğitim döngüsünü başlatma
num_epochs = 8
for epoch in range(num_epochs):
    loss, accuracy = train(model, train_loader, criterion, optimizer, device)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss:.4f}, Accuracy: {accuracy:.2f}%")


  0%|          | 0/485 [00:00<?, ?it/s]

Epoch 1/8, Loss: 0.0969, Accuracy: 96.51%


  0%|          | 0/485 [00:00<?, ?it/s]

In [ ]:
# Modeli kaydetme
torch.save(model.state_dict(), '/content/drive/My Drive/tinyvit_model1.pth')
print("Model başarıyla kaydedildi.")


In [ ]:
# Değerlendirme fonksiyonu
def evaluate(model, val_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

# Değerlendirme
val_accuracy = evaluate(model, val_loader, device)
print(f"Validation Accuracy: {val_accuracy:.2f}%")


In [ ]:
# Test verisi ile değerlendirme
test_accuracy = evaluate(model, test_loader, device)
print(f"Test Accuracy: {test_accuracy:.2f}%")


In [ ]:
#parametre değerlerin bak modellerin hugging face.Patchsize 1 den başla büyüte büyüte git.Gpu kullanımı 2 nin katı.check

In [ ]:
# 7. Modeli Kaydetme
model_save_path = "/content/drive/MyDrive/tinyvit_custom1.pth"
torch.save(model.state_dict(), model_save_path)
print(f"Model kaydedildi: {model_save_path}")